In [4]:
import numpy as np
from scipy.stats import chi2
from random import random, gauss, gammavariate
from tabulate import tabulate
from math import exp, factorial

In [3]:
## Funciones generales (Test para datos discretos)
def calcular_T(N, p):
    """
    Calcula el estadistico T dado en el test chi-cuadrado de 
    Pearson.
    ¡Ambas entradas deben estar ordenadas de menor a mayor respecto
    al valor de variable aleatoria al que hacen referencia!
    Args:
        N:    Frecuencia observada de la muestra
        p:    Probabilidad de los valores posibles de la variable aleatoria
    """
    n = sum(N)    # Tamaño de la muestra
    T = sum((N - n*p)**2 / (n*p))
    return T


def p_valor_pearson(t, grados_libertad):
    """
    Calcula el p-valor usando los resultados teoricos que dictan:
    Ph0(T >= t) = P(X_k-1 >= t)
    Args:
        t              :    Resultado del estadistico en la muestra
        grados_libertad:    Grados de libertad de la variable chi-cuadrada
    """
    acumulada_chi = chi2.cdf(t, grados_libertad)
    p_valor = 1 - acumulada_chi
    return p_valor

def generar_binomial(N, P):
    """
    Genera un valor de la variable aleatoria binomial usando el metodo de la 
    transformada inversa
    Args:
        N: Cantidad de ensayos independientes
        P: Probabilidad de exito de ensayo
    """
    prob = (1 - P) ** N
    c = P / (1 - P)
    F = prob
    i = 0
    U = random()
    while U >= F:
        prob *= c * (N-i) / (i+1)
        F += prob
        i += 1
    return i

def generar_frecuencia_observada(n, p_masa, k):
    """
    Genera la frecuencia observada usando una variable aleatoria binomial
    Args:
        n     :   Tamaño de la muestra
        p_masa:   Probabilidad masa de los valores posibles de la distribucion 
                  de la hipotesis nula
        k     :   Cantidad total de valores posibles de la variable con distribucion
                  dada en la hipotesis nula
    """
    N = np.zeros(k)
    N[0] = generar_binomial(n, p_masa[0])
    for i in range(1, k-1):
        N[i] = generar_binomial(
            n - sum(N[:i]), p_masa[i] / (1 - sum(p_masa[:i])))
    N[k-1] = n - sum(N[:k-1])
    return np.array(N)


def estimar_p_valor_pearson(t, Nsim, p, k, n):
    """
    Estima el p_valor por medio de una simulacion P(T >= t)
    Args:
        t   :   Valor del estadistico en la muestra original
        Nsim:   Cantidad de simulaciones a realizar
        p   :   Probabilidad masa de los valores posibles de la distribucion 
                de la hipotesis nula
        k   :   Cantidad total de valores posibles de la variable con distribucion
                dada en la hipotesis nula
        n   :   Tamaño de la muestra original
    """
    p_valor = 0
    for _ in range(Nsim):
        # Frecuencia observada en la simulacion
        N_sim = generar_frecuencia_observada(n, p, k)
        # Estadistico de la simulacion
        t_sim = calcular_T(N_sim, p)
        if t_sim >= t:
            p_valor += 1
    return p_valor / Nsim

In [24]:
def poisson(x):
    if x < 0:
        return 0
    else:
        return exp(-3) * (3**x) / factorial(x)

muestra = np.array([0, 2, 5, 2, 2, 6, 3, 1, 2, 2, 4, 5, 4, 5, 1, 3, 3, 6, 4, 1])
k = 7
n = 20
N = np.zeros(k)
for dato in muestra:
    N[dato] += 1
p = np.array(list(map(lambda x: poisson(x), list(range(0, k)))))

t = calcular_T(N, p)

# Prueba de pearson
p_valor = p_valor_pearson(t, grados_libertad=k - 1)

# Simulacion
Nsim = 10**4
p_valor_est = estimar_p_valor_pearson(t, Nsim, p, k, n)

data = [[p_valor, p_valor_est, t]]
headers = ["P-valor Pearson", "P-valor simulacion", "Estadistico"]
print(tabulate(data, headers=headers, tablefmt="pretty"))

+--------------------+--------------------+-------------------+
|  P-valor Pearson   | P-valor simulacion |    Estadistico    |
+--------------------+--------------------+-------------------+
| 0.9156224132286204 |       0.9408       | 2.043843888440341 |
+--------------------+--------------------+-------------------+
